# Example

This notebook shows how to train neural network in python in a way that is compatible with the C library.
It does not show how to solve any specific problem, but rather how to transfer model to C.

In [1]:
import numpy as np

In [2]:
# this neural network works the same way as the one in C

class NeuralNetwork:
    def __init__(self, shape: np.array, genotype: np.array):
        self.activation = np.tanh
        self.w = []
        self.genotype = genotype


        parameters_n = 0
        for i in range(len(shape) - 1):
            parameters_n += shape[i] * shape[i+1]
            parameters_n += shape[i+1]

        assert len(self.genotype) == parameters_n, f"genotype: {len(self.genotype)}, parameters: {parameters_n}"
            
        for i in range(len(shape) - 1):
            self.w.append(genotype[:shape[i] * shape[i+1]].reshape(shape[i + 1], shape[i]))
            genotype = genotype[shape[i] * shape[i+1]:]

            self.w.append(genotype[:shape[i+1]])
            self.w[-1] = self.w[-1].reshape(shape[i+1], 1)
            genotype = genotype[shape[i+1]:]

        
    def forward(self, inputs):
        inputs = inputs.reshape(1, -1).T
        for i in range(0, len(self.w), 2):
            inputs = self.w[i] @ inputs
            inputs += self.w[i + 1]
            inputs = self.activation(inputs)
        
        return inputs

In [3]:
# Let's say we want to solve some problem by implementing genetic algorithm.
# It is done by finding the best "genotype" of the function.
# In this case, our function is neural network.

# Create a random genotype
genotype = np.random.randn(170)
# In case you are wondering how do I know that the genotype has to have 170 elements:
# neural network class checks if genotype can fill all weights and biases and tells you if there is wrong number of elements.

# Create a nn with that genotype
layers = [10, 8, 6, 4]
nn = NeuralNetwork(np.array(layers), genotype)


# If neural network with that genotype is doing well, you can save the genotype to a genotype.csv and layers to layers.csv.
with open('genotype.csv', 'w') as f:
    np.savetxt(f, genotype)

with open('layers.csv', 'w') as f:
    for layer in layers[:-1]:
        f.write(str(layer) + ',')
    f.write(str(layers[-1]))

# Of course, this genotype will not be good, because it was generated randomly.
# You will have to implement some sort of learning algorithm.

In [4]:
# If you want to generate neural network using generated genotype, run converter.py.
# It will generate neural_network.h and neural_network.c files.

# Next you can run C code by typing "./run.sh" in terminal (on macOS and Linux).

In [5]:
# now you can generate random inputs to check if C code prints the same output as Python code
inputs = np.random.randn(layers[0])
print(f"float inputs[{layers[0]}] = {{{repr(inputs)[7:-2]}}};")  # paste this as inputs array in C code
print(nn.forward(inputs))

float inputs[10] = { 0.07656088, -0.27443325, -0.12742866, -1.0739909 , -2.3203679 ,
        0.47109858,  0.96793253, -1.20658401, -0.45183336, -0.85693123};
[[ 0.58577476]
 [ 0.99360388]
 [-0.86579222]
 [-0.99730407]]
